In [1]:
import pandas as pd
import psycopg2
import pymysql

detalles = pd.read_excel(r'C:\Users\roquispec\OneDrive - LUZ DEL SUR S.A.A\Documentos\Estudios de Ingreso\ProyectoRyD_V2\Basededatos\detalles_transformadores.xlsx')

In [2]:
import pandas as pd
import pymysql

try:
    connection = pymysql.connect(
        host='vpce-01b25de29c6016b7e-i5efd99f.vpce-svc-074a6d8334ec25452.us-east-1.vpce.amazonaws.com',
        user='USU_SS_RO_ROQUISPEC',
        password='9Y8cE%wjd_',
        database='CDC',
        port=3306
    )
    print("✅ Conexión a SingleStore establecida correctamente")
    
    # CONSULTA SQL PARA EXTRAER DATOS
    consulta_sql = """
SELECT cod_scada, fecha_horita, VEX
FROM (
    SELECT 
        cod_scada,
        DATE(fecha_hora) AS fecha_horita, 
        SUM(comparacion) / 96 AS comparacion_sum,
        SUM(SUM(comparacion) / 96) OVER (PARTITION BY cod_scada ORDER BY DATE(fecha_hora)) AS comparacion_acumulada,
        SUM(SUM(comparacion) / 96) OVER (PARTITION BY cod_scada ORDER BY DATE(fecha_hora)) / (40 * 365) * 100 AS vex_acumulado,
        CASE 
            WHEN SUM(SUM(comparacion) / 96) OVER (PARTITION BY cod_scada ORDER BY DATE(fecha_hora)) / (40 * 365) * 100 > 100 THEN 5
            WHEN SUM(SUM(comparacion) / 96) OVER (PARTITION BY cod_scada ORDER BY DATE(fecha_hora)) / (40 * 365) * 100 > 75 THEN 4
            WHEN SUM(SUM(comparacion) / 96) OVER (PARTITION BY cod_scada ORDER BY DATE(fecha_hora)) / (40 * 365) * 100 > 50 THEN 3
            WHEN SUM(SUM(comparacion) / 96) OVER (PARTITION BY cod_scada ORDER BY DATE(fecha_hora)) / (40 * 365) * 100 > 25 THEN 2
            ELSE 1
        END AS VEX
    FROM (
        SELECT 
            cod_scada, 
            celda, 
            fecha_hora, 
            frecuencia, 
            pot_aparente, 
            tens_prom_s,
            (tens_prom_s / 210) * (60 / frecuencia) * 100 AS lado_izquierdo,
            110 - 5 * pot_aparente / 60 AS lado_derecho,
            CASE 
                WHEN (tens_prom_s / 210) * (60 / frecuencia) * 100 > (110 - 5 * pot_aparente / 60) THEN 1
                ELSE 0
            END AS comparacion
        FROM scada_lectura
        WHERE celda IN ('TRA1', 'TRA2', 'TRA3', 'TRA4')
          AND cod_scada LIKE '%/220/%'
          AND frecuencia IS NOT NULL AND frecuencia != 0
          AND pot_aparente IS NOT NULL AND pot_aparente != 0
          AND tens_prom_s IS NOT NULL AND tens_prom_s != 0
    ) AS base
    GROUP BY cod_scada, DATE(fecha_hora)
) AS resultado
ORDER BY fecha_horita DESC, cod_scada ASC;
    """
    
    # Ejecutar consulta y cargar en DataFrame
    df = pd.read_sql(consulta_sql, connection)
    print(f"✅ Datos extraídos: {len(df)} registros")
    
    # Mostrar primeras filas
    print(df.head())
    
    # Guardar como CSV si necesitas
    # df.to_csv('datos_extraidos.csv', index=False)
    
except pymysql.Error as e:
    print(f"❌ Error de conexión a SingleStore: {e}")
    
except Exception as e:
    print(f"❌ Error inesperado: {e}")
    
finally:
    if 'connection' in locals() and connection.open:
        connection.close()
        print("🔒 Conexión cerrada")

✅ Conexión a SingleStore establecida correctamente


C:\Users\roquispec\AppData\Local\Temp\ipykernel_3460\910065759.py:58: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(consulta_sql, connection)


✅ Datos extraídos: 1834 registros
           cod_scada fecha_horita  VEX
0      ASIA/220/TRA1   2025-10-09    1
1  BALNEARI/220/TRA2   2025-10-09    1
2  BALNEARI/220/TRA3   2025-10-09    1
3  BALNEARI/220/TRA4   2025-10-09    1
4   CANTERA/220/TRA1   2025-10-09    1
🔒 Conexión cerrada


In [3]:
df = df.rename(columns={'fecha_horita': 'FECHA'})
df

,cod_scada,FECHA,VEX
0,ASIA/220/TRA1,2025-10-09,1
1,BALNEARI/220/TRA2,2025-10-09,1
2,BALNEARI/220/TRA3,2025-10-09,1
3,BALNEARI/220/TRA4,2025-10-09,1
4,CANTERA/220/TRA1,2025-10-09,1
...,...,...,...
1829,SANJUAN/220/TRA1,2025-06-01,1
1830,SANJUAN/220/TRA2,2025-06-01,1
1831,SANLUIS/220/TRA2,2025-06-01,1
1832,SROSAN/220/TRA3,2025-06-01,1


In [4]:
# Separar cod_scada en SUBESTACION, TENSION y CIRCUITO
df[['SUBESTACION', 'TENSION', 'CIRCUITO']] = df['cod_scada'].str.split('/', expand=True)

# Reordenar columnas si lo deseas
df = df[['cod_scada', 'SUBESTACION', 'TENSION', 'CIRCUITO', 'FECHA', 'VEX']]
df = df.drop(columns=['cod_scada'])

In [5]:
df

,SUBESTACION,TENSION,CIRCUITO,FECHA,VEX
0,ASIA,220,TRA1,2025-10-09,1
1,BALNEARI,220,TRA2,2025-10-09,1
2,BALNEARI,220,TRA3,2025-10-09,1
3,BALNEARI,220,TRA4,2025-10-09,1
4,CANTERA,220,TRA1,2025-10-09,1
...,...,...,...,...,...
1829,SANJUAN,220,TRA1,2025-06-01,1
1830,SANJUAN,220,TRA2,2025-06-01,1
1831,SANLUIS,220,TRA2,2025-06-01,1
1832,SROSAN,220,TRA3,2025-06-01,1


In [6]:
subestaciones = df['SUBESTACION'].unique().tolist()
subestaciones

['ASIA',
 'BALNEARI',
 'CANTERA',
 'MANCHAY',
 'PACHACUT',
 'PROGRESO',
 'SANJUAN',
 'SANLUIS',
 'SROSAN']

In [7]:
mapeo_subestaciones = {
    'ASIA': 'Asia',
    'BALNEARI': 'Balnearios',
    'CANTERA': 'Cantera',
    'MANCHAY': 'Manchay',
    'PACHACUT': 'Pachacútec',
    'PROGRESO': 'Progreso',
    'SANJUAN': 'San Juan',
    'SANLUIS': 'San Luis',
    'SROSAN': 'Santa Rosa Nueva'
}

In [8]:
df['SUBESTACION'] = df['SUBESTACION'].map(mapeo_subestaciones)
df

,SUBESTACION,TENSION,CIRCUITO,FECHA,VEX
0,Asia,220,TRA1,2025-10-09,1
1,Balnearios,220,TRA2,2025-10-09,1
2,Balnearios,220,TRA3,2025-10-09,1
3,Balnearios,220,TRA4,2025-10-09,1
4,Cantera,220,TRA1,2025-10-09,1
...,...,...,...,...,...
1829,San Juan,220,TRA1,2025-06-01,1
1830,San Juan,220,TRA2,2025-06-01,1
1831,San Luis,220,TRA2,2025-06-01,1
1832,Santa Rosa Nueva,220,TRA3,2025-06-01,1


In [9]:
df= df[['SUBESTACION','CIRCUITO','TENSION','FECHA','VEX']]
df

,SUBESTACION,CIRCUITO,TENSION,FECHA,VEX
0,Asia,TRA1,220,2025-10-09,1
1,Balnearios,TRA2,220,2025-10-09,1
2,Balnearios,TRA3,220,2025-10-09,1
3,Balnearios,TRA4,220,2025-10-09,1
4,Cantera,TRA1,220,2025-10-09,1
...,...,...,...,...,...
1829,San Juan,TRA1,220,2025-06-01,1
1830,San Juan,TRA2,220,2025-06-01,1
1831,San Luis,TRA2,220,2025-06-01,1
1832,Santa Rosa Nueva,TRA3,220,2025-06-01,1


In [10]:
# Unir los DataFrames por SUBESTACION y CIRCUITO
df_resultado = pd.merge(df, detalles, 
                        on=['SUBESTACION', 'CIRCUITO'], how='inner')

# Seleccionar solo las columnas deseadas
df_final = df_resultado[['SERIE', 'FECHA', 'VEX']]

In [11]:
df_final

,SERIE,FECHA,VEX
0,551873,2025-10-09,1
1,551875,2025-10-09,1
2,551874,2025-10-09,1
3,16625S,2025-10-09,1
4,16631S,2025-10-09,1
...,...,...,...
4187,LP-000475,2025-06-01,1
4188,V101043,2025-06-01,1
4189,59571,2025-06-01,1
4190,59572,2025-06-01,1


In [12]:
df_final['SERIE'].unique()

array([551873, 551875, 551874, '16625S', '16631S', '16627S', 185634,
       185635, 185636, 'P186248', 'P186249', 'P186250', 185519, 100138,
       'V101042', 'LP-000473', 'LP-000476', 'V163201', 'V147105',
       '1ZBR60816', '1ZBR60817', '1ZBR60818', 185767, 185766, 185765,
       'V147106', 'LP-000474', 'LP-000475', 'V101043', 59571, 59572,
       59573], dtype=object)

In [13]:
detalles[detalles['SUBESTACION'] == 'Pachacútec']

,SERIE,SUBESTACION,CIRCUITO,FASE,TENSION
27,V147101,Pachacútec,TRA1,R,220/60/10
28,V147102,Pachacútec,TRA1,S,220/60/10
29,V147103,Pachacútec,TRA1,T,220/60/10
55,V163201,Pachacútec,TRA2,NaN,220/10
140,V147104,Pachacútec,Reserva,NaN,220/60/10
